# Dusty Torus Model

This notebook first visualises the `clumpy` models and saves a kindof arbitrary torus template to use with AGNFinder. We then save methods to expose this model as a dill.

In [ ]:
import os
import dill
import h5py
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
from scipy.interpolate import interp2d

import agnfinder
import agnfinder.config as cfg
# change working directory to root of `agnfinder` project
os.chdir(os.path.join(os.getcwd(), "../"))
os.getcwd()

In [ ]:
sns.set_context('notebook')

In [ ]:
# This needs to be specified in cfg.TorusConfiguration or whatever.
# Question: what is in this data set?
# If generated on a grid, can we generate it ourselves rather than having
# to download a > ~1Gb file every time?

data_loc = os.path.join(os.getcwd(), "data/clumpy_models_201410_tvavg.hdf5")

In [ ]:
with h5py.File(data_loc, 'r') as f:
    print(f.keys())
    wavelengths = f['wave'][...] * 1e4  # microns to angstroms
    opening_angle = f['sig'][...]
    inclination = f['i'][...]
    n0 = f['N0'][...]
    q = f['q'][...]
    y = f['Y'][...]
    tv = f['tv'][...]
    seds = f['flux_toragn'][...]

In [ ]:
seds.shape

In [ ]:
plt.hist(opening_angle, bins=100)
plt.show()

Clearly the opening angles of toruses (tori?) are not quantised like this in the real world; these parameters have been generated on a grid.

MW (and SF) fix this angle to 30deg.

1. Is 30deg completely arbitrary?
2. What are the consequences of fixing this arbitrarily?
3. Could we instead allow it to vary (with some sensible prior)?

In [ ]:
def get_log_space_median(x):
    return 10 ** np.median(np.log10(x), axis=0)

In [ ]:
median_sed = get_log_space_median(seds)
plt.loglog(wavelengths, median_sed, 'r', label='Median (log-space)')
random_indices = np.random.choice(len(seds), 500)
random_seds = seds[random_indices]
for s in random_seds:
    plt.loglog(wavelengths, s, color='k', alpha=0.01)
plt.xlabel('Wavelength')
plt.ylabel('Normalised Flux')
plt.ylim([10**-5, 1])
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Boolean mask
print(opening_angle)
print(opening_angle.shape)
desired_opening_angle = opening_angle == 30
print(desired_opening_angle)
print(desired_opening_angle.shape)

In [ ]:
print(inclination.shape)
print(inclination)
unique_inclinations = np.unique(inclination[desired_opening_angle])
print(unique_inclinations.shape)
print(unique_inclinations)

## What do the SEDs look like?

In particular, we want to know what effect inclination has, for the opening angle. Here we have selected the opening angle of 30deg, and we are varying the inclination (denoted by the different line colours) as well as marginalising the log median over the other parameters.

In [ ]:
for inc in unique_inclinations:
    plt.loglog(wavelengths, get_log_space_median(seds[desired_opening_angle & (inclination == inc)]), color=cm.plasma(inc / unique_inclinations.max()))
plt.loglog(wavelengths, get_log_space_median(seds), 'k-.', label=r'Log Median (All)')
plt.legend()
plt.xlim([10**3, None])
plt.ylim([10**-3, 1])
plt.xlabel('Wavelength')
plt.ylabel('Flux')                                   

From the plot above, we can see that the inclination has an effect on the 'break' around $10^4$.

We now pick some fixed parameters using values which are assumed in the simulation author's paper, when varying one parameter and fixing the rest.

In [ ]:
figure_suggested_params = (n0 == 5) & (opening_angle == 30) & (q == 2) & (y == 30) & (tv == 60)
figure_suggested_params.sum()  # number of samples satisfying these constraints

In [ ]:
pd.unique(inclination[figure_suggested_params])

Hence we have 10 possibilities, one for each inclination value. If we step through them, what do they look like

In [ ]:
unique_inclinations = inclination[figure_suggested_params]
for inc in unique_inclinations:
    plt.loglog(wavelengths, 
               get_log_space_median(seds[figure_suggested_params & (inclination == inc)]), 
               color=cm.plasma(inc / unique_inclinations.max()))
plt.xlim([10**3, None])
plt.ylim([10**-3, 1])
plt.xlabel('Wavelength')
plt.ylabel('Flux')

At a glance, these two plots seem quite similar. Fixing the other parameters doesn't seem to have made as large an effect as fixing the inclination would have (which accounts for most of the variation in Flux)

## Creating the Torus Model

We will make these 10 SEDs (10 because of the parameter grid) into a smooth callable function of (wavelength, inclination), interpolating in log space. This effectively places a log-uniform prior on these values.

In [ ]:
func = interp2d(x=np.log10(wavelengths), 
                y=inclination[figure_suggested_params],
                z=np.log10(seds[figure_suggested_params]))

In [ ]:
# Save the model to dill:
with open(cfg.QuasarTemplateParams.torus_model_loc, 'wb') as file:
    dill.dump(func, file)

Plot the interpolated model (`func`)

In [ ]:
import matplotlib as mpl
sns.set_style('ticks')
x = np.log10(wavelengths)
# Set the possible inclination values to[0, 90]
inclinations_y = np.linspace(0, 90, 300)
for y in inclinations_y:
    z = func(x=x, y=y)
    plt.loglog(10**x, 10**z, color=cm.plasma(y / inclinations_y.max()))
plt.xlim([10**4, 10**6])
plt.ylim([10**-2, 1])
plt.xlabel('Wavelength (A)')
plt.ylabel('Flux (normalised)')

cbar = plt.colorbar(cm.ScalarMappable(cmap=cm.plasma, norm = mpl.colors.Normalize(vmin=0.,vmax=inclinations_y.max())))
cbar_label = r'Inclination ($\deg$)'
cbar.ax.get_yaxis().labelpad=25
cbar.ax.set_ylabel(cbar_label, rotation=270)
plt.tight_layout()